In [0]:
configs = {"fs.azure.account.auth.type": "OAuth",
"fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
"fs.azure.account.oauth2.client.id": "",
"fs.azure.account.oauth2.client.secret": '',
"fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/tenantid/oauth2/token"}

dbutils.fs.mount(
source = "abfss://contrainer@storageacc.dfs.core.windows.net", # contrainer@storageacc
mount_point = "/mnt/bronze",
extra_configs = configs)


In [0]:
configs = {"fs.azure.account.auth.type": "OAuth",
"fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
"fs.azure.account.oauth2.client.id": "",
"fs.azure.account.oauth2.client.secret": '',
"fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/tenantid/oauth2/token"}

dbutils.fs.mount(
source = "abfss://contrainer@storageacc.dfs.core.windows.net", # contrainer@storageacc
mount_point = "/mnt/silver",
extra_configs = configs)



Out[20]: True

In [0]:
configs = {"fs.azure.account.auth.type": "OAuth",
"fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
"fs.azure.account.oauth2.client.id": "",
"fs.azure.account.oauth2.client.secret": '',
"fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/tenantid/oauth2/token"}

dbutils.fs.mount(
source = "abfss://contrainer@storageacc.dfs.core.windows.net", # contrainer@storageacc
mount_point = "/mnt/gold",
extra_configs = configs)



Out[22]: True

In [0]:
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.2 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
table_name=[]
for i in dbutils.fs.ls("/mnt/bronze/SalesLT/"):
    table_name.append(i.name.split('/')[0])

In [0]:
for i in table_name:
    path='/mnt/bronze/SalesLT/' + i + '/' + i + '.parquet'
    df=spark.read.format('parquet').load(path)
    column = df.columns
    for col in column:
        if "Date" in col or "date" in col:
            df = df.withColumn(col, date_format(from_utc_timestamp(df[col].cast(TimestampType()), "UTC"), "yyyy-MM-dd"))
        
    output_path = '/mnt/silver/SalesLT/' +i +'/'
    df.write.format('delta').mode("overwrite").save(output_path)